In [59]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup as bs
from time import sleep

In [60]:
# set up the webdriver
driver = webdriver.Chrome()

In [61]:
# go to webpage
driver.get('https://www.indeed.com/jobs?q=cyber%20security&l=United%20States')

In [62]:
#let the driver wait 3 seconds to locate the element before exiting out
driver.implicitly_wait(3) 

In [27]:
#close_popup = driver.find_element_by_id("popover-x-button-close")
#close_popup.click()

In [28]:
titles=[]
companies=[]
locations=[]
links =[]
#reviews=[]
salaries = []
descriptions=[]

for i in range(0,29):
    
    job_card = driver.find_elements_by_xpath('//div[contains(@class,"job_seen_beacon")]')

    r = requests.get(driver.current_url)
    soup = bs(r.content, 'html')
    
    for job, job_link in zip(job_card, soup.select('.result')):
        links.append(f'https://indeed.com/viewjob?jk={job_link["data-jk"]}')
       
    #.   not all positions have salary
        try:
            salary = job.find_element_by_xpath(".//span[@class='salary-snippet']").text
            
        except:
            salary = "None"
    #.  tells only to look at the element       
        salaries.append(salary)
        
        try:
            location = job.find_element_by_xpath(".//div[@class='companyLocation']").text
        except:
            location = "None"
    #.  tells only to look at the element       
        locations.append(location)
        
        try:
            title  = job.find_element_by_xpath(".//h2[@class='jobTitle jobTitle-color-purple']").text
        except:
            title = "None"
        titles.append(title)
        #links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))
        companies.append(job.find_element_by_xpath(".//span[@class='companyName']").text)
        
       
        
    
    try:
        next_page = driver.find_element_by_xpath('//a[@aria-label={}]//span[@class="pn"]'.format(i+2))
        next_page.click()

    except:
        next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="np"]')
        next_page.click()
    #except:
        #next_page = driver.find_element_by_xpath('//a[.//span[contains(text(),"Next")]]')
        #next_page.click()
        
    
    print("Page: {}".format(str(i+2)))

Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30


In [29]:
# validate that code for links properly looped and isn't repeating
import pandas as pd
pd.Series(links).value_counts()

https://indeed.com/viewjob?jk=67c6aff1436b7935    4
https://indeed.com/viewjob?jk=3d76d9919c91de5b    3
https://indeed.com/viewjob?jk=18c0e9e20c082608    3
https://indeed.com/viewjob?jk=21a8c2af9eaee652    3
https://indeed.com/viewjob?jk=fa1c10002c08caab    3
                                                 ..
https://indeed.com/viewjob?jk=72f52260907114b4    1
https://indeed.com/viewjob?jk=8592e7ce35b6fddb    1
https://indeed.com/viewjob?jk=db816ef3bede62d1    1
https://indeed.com/viewjob?jk=ffd68d7b7bfeb8ed    1
https://indeed.com/viewjob?jk=92424678cd39c641    1
Length: 273, dtype: int64

In [30]:
# validate all links are there
len(links)

332

In [63]:
# loop through links and bring in the job descpriptions
descriptions=[]
for link in links:
    
    driver.get(link)
    jd = driver.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
    descriptions.append(jd)
    sleep(15)

In [67]:
# create the dataset 

indeed_df = pd.DataFrame()
indeed_df['Title'] = titles
indeed_df['Company'] = companies
indeed_df['Location'] = locations
indeed_df['Links'] = links
indeed_df['Salary'] = salaries
indeed_df['Description'] = descriptions

In [68]:
# validate dataset was created properly
indeed_df.head()

,Title,Company,Location,Links,Salary,Description
0,Senior Software Engineer Security and Authenti...,MathWorks,"Natick, MA 01760",https://indeed.com/viewjob?jk=dedaae73a1bb2555,None,We are looking for a highly motivated work-fro...
1,None,Citizens,"Greenville, RI 02828",https://indeed.com/viewjob?jk=8b0ad10599f3a304,None,Job Description:\nAre you looking for a new an...
2,"2021745 Cyber Security Analysts $215,000.00",B4CORP,"Reston, VA",https://indeed.com/viewjob?jk=407b2677aa6ccc25,"Up to $215,000 a year",US CITIZENSHIP REQUIRED FOR THIS POSITION: Yes...
3,Senior Cyber Security Analyst,TherapyNotes.com,Remote,https://indeed.com/viewjob?jk=0dd5712bac6c57aa,None,"Cyber Security Analyst, Incident Response, Cyb..."
4,Cyber Security Analyst,NDK,"Morrisville, NC•Remote",https://indeed.com/viewjob?jk=2d39ed3a3baf0863,"Up to $130,000 a year",Information Security Engineer\nThe Information...


In [69]:
# export to csv (UPDATE THE FILE TITLE)
indeed_df.to_csv('indeed_dataset.csv', index = False)